# Train Test Split and Cross Validation


## Imports

As with other notebooks we will use the same set of standard imports.

In [7]:
import numpy as np
import pandas as pd

In [8]:
import plotly.offline as py
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.figure_factory as ff
import cufflinks as cf
cf.set_config_file(offline=True, sharing=False, theme='ggplot');

In [9]:
from sklearn.linear_model import LinearRegression

## The Data

For this notebook, we will use the seaborn `mpg` dataset which describes the fuel mileage (measured in miles per gallon or mpg) of various cars along with characteristics of those cars.  Our goal will be to build a model that can predict the fuel mileage of a car based on the characteristics of that car.

In [10]:
from seaborn import load_dataset
data = load_dataset("mpg")
data

mpg  cylinders  displacement  horsepower  weight  acceleration  \
0    18.0          8         307.0       130.0    3504          12.0   
1    15.0          8         350.0       165.0    3693          11.5   
2    18.0          8         318.0       150.0    3436          11.0   
3    16.0          8         304.0       150.0    3433          12.0   
4    17.0          8         302.0       140.0    3449          10.5   
..    ...        ...           ...         ...     ...           ...   
393  27.0          4         140.0        86.0    2790          15.6   
394  44.0          4          97.0        52.0    2130          24.6   
395  32.0          4         135.0        84.0    2295          11.6   
396  28.0          4         120.0        79.0    2625          18.6   
397  31.0          4         119.0        82.0    2720          19.4   

     model_year  origin                       name  
0            70     usa  chevrolet chevelle malibu  
1            70     usa          buick skylark 320  
2            70     usa         plymouth satellite  
3            70     usa              amc rebel sst  
4            70     usa                ford torino  
..          ...     ...                        ...  
393          82     usa            ford mustang gl  
394          82  europe                  vw pickup  
395          82     usa              dodge rampage  
396          82     usa                ford ranger  
397          82     usa                 chevy s-10  

[398 rows x 9 columns]

## Train Test Split

The first thing we will want to do with this data is construct a train/test split. Constructing a train test split before EDA and data cleaning can often be helpful.  This allows us to see if our data cleaning and any conclusions we draw from visualizations generalize to new data. This can be done by re-running the data cleaning and EDA process on the test dataset.

### Using Pandas Operations

We can sample the entire dataset to get a permutation and then select a range of rows.

In [11]:
shuffled_data = data.sample(frac=1., random_state=42)
shuffled_data

mpg  cylinders  displacement  horsepower  weight  acceleration  \
198  33.0          4          91.0        53.0    1795          17.4   
396  28.0          4         120.0        79.0    2625          18.6   
33   19.0          6         232.0       100.0    2634          13.0   
208  13.0          8         318.0       150.0    3940          13.2   
93   14.0          8         318.0       150.0    4237          14.5   
..    ...        ...           ...         ...     ...           ...   
71   19.0          3          70.0        97.0    2330          13.5   
106  12.0          8         350.0       180.0    4499          12.5   
270  21.1          4         134.0        95.0    2515          14.8   
348  37.7          4          89.0        62.0    2050          17.3   
102  26.0          4          97.0        46.0    1950          21.0   

     model_year  origin                        name  
198          76   japan                 honda civic  
396          82     usa                 ford ranger  
33           71     usa                 amc gremlin  
208          76     usa  plymouth volare premier v8  
93           73     usa    plymouth fury gran sedan  
..          ...     ...                         ...  
71           72   japan             mazda rx2 coupe  
106          73     usa    oldsmobile vista cruiser  
270          78   japan   toyota celica gt liftback  
348          81   japan               toyota tercel  
102          73  europe     volkswagen super beetle  

[398 rows x 9 columns]

Selecting a range of rows for training and test

In [12]:
split_point = int(shuffled_data.shape[0]*0.90)
split_point

358

In [13]:
tr = shuffled_data.iloc[:split_point]
te = shuffled_data.iloc[split_point:]

In [14]:
tr

mpg  cylinders  displacement  horsepower  weight  acceleration  \
198  33.0          4          91.0        53.0    1795          17.4   
396  28.0          4         120.0        79.0    2625          18.6   
33   19.0          6         232.0       100.0    2634          13.0   
208  13.0          8         318.0       150.0    3940          13.2   
93   14.0          8         318.0       150.0    4237          14.5   
..    ...        ...           ...         ...     ...           ...   
391  36.0          4         135.0        84.0    2370          13.0   
134  16.0          6         258.0       110.0    3632          18.0   
306  28.8          6         173.0       115.0    2595          11.3   
381  36.0          4         107.0        75.0    2205          14.5   
319  31.3          4         120.0        75.0    2542          17.5   

     model_year origin                        name  
198          76  japan                 honda civic  
396          82    usa                 ford ranger  
33           71    usa                 amc gremlin  
208          76    usa  plymouth volare premier v8  
93           73    usa    plymouth fury gran sedan  
..          ...    ...                         ...  
391          82    usa           dodge charger 2.2  
134          74    usa                 amc matador  
306          79    usa          chevrolet citation  
381          82  japan                honda accord  
319          80  japan                   mazda 626  

[358 rows x 9 columns]

In [15]:
te

mpg  cylinders  displacement  horsepower  weight  acceleration  \
243  21.5          3          80.0       110.0    2720          13.5   
54   35.0          4          72.0        69.0    1613          18.0   
50   28.0          4         116.0        90.0    2123          14.0   
174  18.0          6         171.0        97.0    2984          14.5   
189  15.5          8         304.0       120.0    3962          13.9   
395  32.0          4         135.0        84.0    2295          11.6   
187  17.5          8         305.0       140.0    4215          13.0   
169  20.0          6         232.0       100.0    2914          16.0   
58   25.0          4          97.5        80.0    2126          17.0   
48   18.0          6         250.0        88.0    3139          14.5   
344  39.0          4          86.0        64.0    1875          16.4   
235  26.0          4          97.0        75.0    2265          18.2   
252  19.2          6         231.0       105.0    3535          19.2   
21   24.0          4         107.0        90.0    2430          14.5   
313  28.0          4         151.0        90.0    2678          16.5   
160  17.0          6         231.0       110.0    3907          21.0   
276  21.6          4         121.0       115.0    2795          15.7   
191  22.0          6         225.0       100.0    3233          15.4   
293  31.9          4          89.0        71.0    1925          14.0   
343  39.1          4          79.0        58.0    1755          16.9   
257  19.4          6         232.0        90.0    3210          17.2   
308  33.5          4         151.0        90.0    2556          13.2   
149  24.0          4         120.0        97.0    2489          15.0   
130  26.0          4         122.0        80.0    2451          16.5   
151  31.0          4          79.0        67.0    2000          16.0   
359  28.1          4         141.0        80.0    3230          20.4   
99   18.0          6         232.0       100.0    2945          16.0   
372  27.0          4         151.0        90.0    2735          18.0   
87   13.0          8         350.0       145.0    3988          13.0   
330  40.9          4          85.0         NaN    1835          17.3   
214  13.0          8         302.0       130.0    3870          15.0   
121  15.0          8         318.0       150.0    3399          11.0   
397  31.0          4         119.0        82.0    2720          19.4   
20   25.0          4         110.0        87.0    2672          17.5   
188  16.0          8         318.0       150.0    4190          13.0   
71   19.0          3          70.0        97.0    2330          13.5   
106  12.0          8         350.0       180.0    4499          12.5   
270  21.1          4         134.0        95.0    2515          14.8   
348  37.7          4          89.0        62.0    2050          17.3   
102  26.0          4          97.0        46.0    1950          21.0   

     model_year  origin                               name  
243          77   japan                         mazda rx-4  
54           71   japan                        datsun 1200  
50           71  europe                          opel 1900  
174          75     usa                         ford pinto  
189          76     usa                        amc matador  
395          82     usa                      dodge rampage  
187          76     usa  chevrolet chevelle malibu classic  
169          75     usa                        amc gremlin  
58           72     usa                 dodge colt hardtop  
48           71     usa                       ford mustang  
344          81     usa                     plymouth champ  
235          77   japan            toyota corolla liftback  
252          78     usa                 pontiac phoenix lj  
21           70  europe                        audi 100 ls  
313          80     usa                 chevrolet citation  
160          75     usa                      buick century  
276          78 

Checking that they add up.

In [16]:
len(tr) + len(te) == len(data)

True

### Using SKLearn

We can use the `train_test_split` function from `sklearn.model_selection` to do this easily.

In [17]:
from sklearn.model_selection import train_test_split

In [18]:
tr, te = train_test_split(data, test_size=0.1, random_state=83)

In [19]:
tr

mpg  cylinders  displacement  horsepower  weight  acceleration  \
143  26.0          4          97.0        78.0    2300          14.5   
391  36.0          4         135.0        84.0    2370          13.0   
182  28.0          4         107.0        86.0    2464          15.5   
63   14.0          8         400.0       175.0    4385          12.0   
52   30.0          4          88.0        76.0    2065          14.5   
..    ...        ...           ...         ...     ...           ...   
394  44.0          4          97.0        52.0    2130          24.6   
256  20.5          6         225.0       100.0    3430          17.2   
295  35.7          4          98.0        80.0    1915          14.4   
23   26.0          4         121.0       113.0    2234          12.5   
82   23.0          4         120.0        97.0    2506          14.5   

     model_year  origin                         name  
143          74  europe                   opel manta  
391          82     usa            dodge charger 2.2  
182          76  europe                     fiat 131  
63           72     usa             pontiac catalina  
52           71  europe                    fiat 124b  
..          ...     ...                          ...  
394          82  europe                    vw pickup  
256          78     usa              plymouth volare  
295          79     usa  dodge colt hatchback custom  
23           70  europe                     bmw 2002  
82           72   japan  toyouta corona mark ii (sw)  

[358 rows x 9 columns]

In [20]:
te

mpg  cylinders  displacement  horsepower  weight  acceleration  \
232  16.0          8         351.0       149.0    4335          14.5   
365  20.2          6         200.0        88.0    3060          17.1   
225  17.5          6         250.0       110.0    3520          16.4   
277  16.2          6         163.0       133.0    3410          15.8   
115  15.0          8         350.0       145.0    4082          13.0   
314  26.4          4         140.0        88.0    2870          18.1   
219  25.5          4         122.0        96.0    2300          15.5   
33   19.0          6         232.0       100.0    2634          13.0   
312  37.2          4          86.0        65.0    2019          16.4   
306  28.8          6         173.0       115.0    2595          11.3   
280  21.5          6         231.0       115.0    3245          15.4   
97   18.0          6         225.0       105.0    3121          16.5   
0    18.0          8         307.0       130.0    3504          12.0   
100  18.0          6         250.0        88.0    3021          16.5   
29   27.0          4          97.0        88.0    2130          14.5   
300  23.9          8         260.0        90.0    3420          22.2   
281  19.8          6         200.0        85.0    2990          18.2   
252  19.2          6         231.0       105.0    3535          19.2   
388  26.0          4         156.0        92.0    2585          14.5   
376  37.0          4          91.0        68.0    2025          18.2   
278  31.5          4          89.0        71.0    1990          14.9   
101  23.0          6         198.0        95.0    2904          16.0   
76   18.0          4         121.0       112.0    2933          14.5   
343  39.1          4          79.0        58.0    1755          16.9   
362  24.2          6         146.0       120.0    2930          13.8   
327  36.4          5         121.0        67.0    2950          19.9   
170  23.0          4         140.0        78.0    2592          18.5   
50   28.0          4         116.0        90.0    2123          14.0   
246  32.8          4          78.0        52.0    1985          19.4   
32   25.0          4          98.0         NaN    2046          19.0   
89   15.0          8         318.0       150.0    3777          12.5   
230  15.5          8         350.0       170.0    4165          11.4   
313  28.0          4         151.0        90.0    2678          16.5   
86   14.0          8         304.0       150.0    3672          11.5   
46   22.0          4         140.0        72.0    2408          19.0   
364  26.6          8         350.0       105.0    3725          19.0   
272  23.8          4         151.0        85.0    2855          17.6   
171  24.0          4         134.0        96.0    2702          13.5   
71   19.0          3          70.0        97.0    2330          13.5   
231  15.5          8         400.0       190.0    4325          12.2   

     model_year  origin                               name  
232          77     usa                   ford thunderbird  
365          81     usa                    ford granada gl  
225          77     usa                 chevrolet concours  
277          78  europe                      peugeot 604sl  
115          73     usa            chevrolet monte carlo s  
314          80     usa                      ford fairmont  
219          77     usa                  plymouth arrow gs  
33           71     usa                        amc gremlin  
312          80   japan                         datsun 310  
306          79     usa                 chevrolet citation  
280          79     usa                  pontiac lemans v6  
97           73     usa                   plymouth valiant  
0            70     usa          chevrolet chevelle malibu  
100          73     usa                      ford maverick  
29           71   japan                       datsun pl510  
300          79     usa  oldsmobile cutlass salon brougham  
281          79 

### Quick Visualization

Out of curiosity what does the `mgp` field look like?  I am going to look at both the train and test distributions but **in practice we should avoid looking at the test data**.

In [21]:
ff.create_distplot([tr['mpg'], te['mpg']], ['train mpg', 'test mpg'])

<br/><br/><br/>


---



## Building A Basic Model

Let's go through the process of building a model.  Let's start by looking at just engine characteristics like "cylinders" and the "displacement". We will first use just our own feature function (as we did in previous lectures).  Then we will introduce how to use sklearn `Pipelines` to combine feature functions and models. As we will see, by combining the feature function and model, we can simplify subsequent training and testing since we are guaranteed that our feature functions are the same on both the training and test datasets. 

My first feature function will just extract the two features that I want to use in my model.

In [22]:
def phi(df):
    return df[["cylinders", "displacement"]]

Then if it an sklearn LinearRegression model to my training data.

In [23]:
from sklearn.linear_model import LinearRegression
model = LinearRegression()

In [24]:
model.fit(phi(tr), tr['mpg'])

LinearRegression()

To evaluate the error we will use the **Root Mean Squared Error (RMSE)** which is like the mean squared error but in the correct units (mpg) instead of (mpg^2). 

In [25]:
def rmse(y, yhat):
    return np.sqrt(np.mean((y - yhat)**2))

The training error is:

In [26]:
Y_hat = model.predict(phi(tr))
Y = tr['mpg']
print("Training Error (RMSE):", rmse(Y, Y_hat))

Training Error (RMSE): 4.589009902639974


### Don't try this at home! 

The test error is:

In [27]:
Y_hat = model.predict(phi(te))
Y = te['mpg']
print("Test Error (RMSE):", rmse(Y, Y_hat))

Test Error (RMSE): 5.034009303909175


Oh no! We just used the test data to evaluate our model! We shouldn't have done that.  

(Don't worry, we are trained professionals and this is only for demonstration purposes.  But seriously, don't try this at home.)  

**Notice:** The test error is slightly higher than the training error.  This is typically (but not always) the case.  Sometimes we get lucky and the test data is "easier to predict" or happens to closely follow the training data.

<br/><br/><br/>

---

## SKLearn Pipelines

Notice that we need to apply the same transformations to the test data when we go to evaluate our model.  Keeping track of phi can get a bit tedious.  We will use the sklearn Pipelines package to combine feature engineering stages and the model into a single "model" that applies the same transformations when you call `fit` and `predict`.  You may want to [skim the documentation on pipelines](https://scikit-learn.org/stable/modules/compose.html#pipeline).

A Pipeline is composed of stages (in a list) where each stage has a name which can be used to access that stage (more on this later) and an operation that must either be a transformation or a model. 

In [28]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In [29]:
model = Pipeline([
    ("SelectColumns", ColumnTransformer([("keep", "passthrough", ["cylinders", "displacement"])])),
    ("LinearModel", LinearRegression())
])

The first stage of this pipeline just selects the columns:

In [30]:
model['SelectColumns']

ColumnTransformer(transformers=[('keep', 'passthrough',
                                 ['cylinders', 'displacement'])])

The [`ColumnTransformer`](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html#sklearn.compose.ColumnTransformer) is designed to capture basic feature engineering operations on pandas DataFrames.  It takes a list of column transformation triples.  Each transformation gets a name (e.g., "keep"), an operation which could be something like `OneHotEncoder`, or a string describing what to do `"drop"` or `"passthrough"` and then finally the list of column names on which to apply the transformation.

We can now fit the **entire pipeline**:

In [31]:
model.fit(tr, tr['mpg'])

Pipeline(steps=[('SelectColumns',
                 ColumnTransformer(transformers=[('keep', 'passthrough',
                                                  ['cylinders',
                                                   'displacement'])])),
                ('LinearModel', LinearRegression())])

Then we can make predictions on the entire DataFrame and error of our model on the training data

In [32]:
Y_hat = model.predict(tr)
Y = tr['mpg']
print("Training Error (RMSE):", rmse(Y, Y_hat))

Training Error (RMSE): 4.589009902639974


Again, we shouldn't do this but I want to illustrate how this model can be easily applied to the test data without having to worry about applying the feature function first.

In [33]:
Y_hat = model.predict(te)
Y = te['mpg']
print("Test Error (RMSE):", rmse(Y, Y_hat))

Test Error (RMSE): 5.034009303909175


#### Keeping track of all the models.

In this notebook (and in life) we will want to keep track of all our models.  Here I will store the models in a dictionary with a (not great) name so I can remember which model is which and can easily compare my models in a plot.

In [34]:
models = {"c+d": model}

### More Feature Transformations

We might also want to look at the displacement per cylinder.  This is an additional feature transformation that we can add to the first stage of our pipeline.  To define this transformation we first need to create a function transformer:

In [35]:
from sklearn.preprocessing import FunctionTransformer

def compute_volume(X):
    return np.expand_dims(X[:,1] / X[:,0]  , axis=1)

volume_transformer = FunctionTransformer(compute_volume, validate=True)

We can then add this as an additional column transformation:

In [36]:
model = Pipeline([
    ("SelectColumns", ColumnTransformer([
        ("keep", "passthrough", ["cylinders", "displacement"]),
        ("cyl_vol", volume_transformer, ["cylinders", "displacement"])])),
    ("LinearModel", LinearRegression())
])

In [37]:
model.fit(tr, tr['mpg'])

Pipeline(steps=[('SelectColumns',
                 ColumnTransformer(transformers=[('keep', 'passthrough',
                                                  ['cylinders',
                                                   'displacement']),
                                                 ('cyl_vol',
                                                  FunctionTransformer(func=<function compute_volume at 0x7fc7e77d7430>,
                                                                      validate=True),
                                                  ['cylinders',
                                                   'displacement'])])),
                ('LinearModel', LinearRegression())])

Again, we evaluate the model on our training dataset and see a reduction in error:

In [38]:
Y_hat = model.predict(tr)
Y = tr['mpg']
print("Training Error (RMSE):", rmse(Y, Y_hat))

Training Error (RMSE): 4.318799994795116


In [39]:
models["c+d+d/c"] = model

### Adding More Features

We can now add additional features about the car.

In [40]:
quantitative_features = ["cylinders", "displacement", "horsepower", "weight", "acceleration"]
model = Pipeline([
    ("SelectColumns", ColumnTransformer([
        ("keep", "passthrough", quantitative_features),
        ("cyl_vol", volume_transformer, ["cylinders", "displacement"])])),
    ("LinearModel", LinearRegression())
])

I have put the following code in a try/except statement because I know it will raise an error. What do you think will go wrong?

In [41]:
try:
    model.fit(tr, tr['mpg'])
except ValueError as err:
    print(err)

Input contains NaN, infinity or a value too large for dtype('float64').


There appear to be NaN (missing values) in the data (take a look at the horsepower column).  We need to deal with these missing values.  In previous lectures I mentioned imputation and a standard imputation technique is to replace the missing value with the mean for that column.  Scikit learn has a built-in imputation function that we can add to our pipeline after we select the desired columns.  The imputation will actually be applied to all the columns.  If we wanted to apply it to a specific column then we would need to put it inside the ColumnTransformer.

**Notice:** The imputation function actually needs to be **fit** to data so it is also part of the model.

In [42]:
from sklearn.impute import SimpleImputer
model = Pipeline([
    ("SelectColumns", ColumnTransformer([
        ("keep", "passthrough", quantitative_features),
        ("cyl_vol", volume_transformer, ["cylinders", "displacement"])])),
    ("Imputation", SimpleImputer()),
    ("LinearModel", LinearRegression())
])

We can now train our model.

In [43]:
model.fit(tr, tr['mpg'])

Pipeline(steps=[('SelectColumns',
                 ColumnTransformer(transformers=[('keep', 'passthrough',
                                                  ['cylinders', 'displacement',
                                                   'horsepower', 'weight',
                                                   'acceleration']),
                                                 ('cyl_vol',
                                                  FunctionTransformer(func=<function compute_volume at 0x7fc7e77d7430>,
                                                                      validate=True),
                                                  ['cylinders',
                                                   'displacement'])])),
                ('Imputation', SimpleImputer()),
                ('LinearModel', LinearRegression())])

Saving the model for later comparison:

In [44]:
models['c+d+d/c+h+w+a'] = model

Evaluating the training error:

In [45]:
Y_hat = model.predict(tr)
Y = tr['mpg']
print("Training Error (RMSE):", rmse(Y, Y_hat))

Training Error (RMSE): 4.025034093127813


We reduced the training error but what about the test error?  We really shouldn't look at the test error so instead we will use cross validation to compare the accuracy:

<br/><br/><br/>

---

## Cross Validation

In the following function we use the sklear `KFold` cross validation class. 

Here we define a five fold cross validation with 

```python 
five_fold = KFold(n_splits=5)
```

Then we loop over the 5 splits and get the indicies (`tr_ind`) in the training data to use for training and the indices (`va_ind`) in the training data to use for validation:

```python
for tr_ind, te_ind in five_fold.split(tr):
```

In [46]:
from sklearn.model_selection import KFold
from sklearn.base import clone

def cross_validate_rmse(model):
    model = clone(model)
    five_fold = KFold(n_splits=5)
    rmse_values = []
    for tr_ind, va_ind in five_fold.split(tr):
        model.fit(tr.iloc[tr_ind,:], tr['mpg'].iloc[tr_ind])
        rmse_values.append(rmse(tr['mpg'].iloc[va_ind], model.predict(tr.iloc[va_ind,:])))
    return np.mean(rmse_values)


Valiating the model

In [47]:
cross_validate_rmse(model)

4.05180814386984

The following helper function generates a plot comparing all the models in the models dictionary.

In [48]:
def compare_models(models):
    # Compute the training error for each model
    training_rmse = [rmse(tr['mpg'], model.predict(tr)) for model in models.values()]
    # Compute the cross validation error for each model
    validation_rmse = [cross_validate_rmse(model) for model in models.values()]
    # Compute the test error for each model (don't do this!)
    test_rmse = [rmse(te['mpg'], model.predict(te)) for model in models.values()]
    names = list(models.keys())
    fig = go.Figure([
        go.Bar(x = names, y = training_rmse, name="Training RMSE"),
        go.Bar(x = names, y = validation_rmse, name="CV RMSE"),
        go.Bar(x = names, y = test_rmse, name="Test RMSE", opacity=.3)])
    return fig

In [49]:
fig = compare_models(models)
fig.update_yaxes(range=[2,5.1], title="RMSE")

Notice I made the **Test RMSE** invisible(ish) because you shouldn't look at it until we are done.  But again for demonstration purposes I plotted in so we can see how it compares to the training and cross validation errors.

Can you improve the model further?  Let's try adding the model year

In [50]:
quantitative_features = ["cylinders", "displacement", "horsepower", "weight", "acceleration", "model_year"]
model = Pipeline([
    ("SelectColumns", ColumnTransformer([
        ("keep", "passthrough", quantitative_features),
        ("cyl_vol", volume_transformer, ["cylinders", "displacement"])
    ])),
    ("Imputation", SimpleImputer()),
    ("LinearModel", LinearRegression())
])

In [51]:
model.fit(tr, tr['mpg'])
models['c+d+d/c+h+w+a+y'] = model

Comparing the models

In [52]:
fig = compare_models(models)
fig.update_yaxes(range=[2,5.1], title="RMSE")

The model year improved accuracy quite a bit!  This improvement also appears to generalize as it also reduced the cross validation error. 

## Going too Far?

Can we use the car's name to predict MPG?  The name contains general features like the manufacturer that might help but it also contains the vehicle make which is probably too specific and not all the vehicles in test will appear in training. 

Let's try by applying the CountVectorizer (which implements the bag of words features).  At this point we are also likely to have too many dimensions in our model and we are not applying any regularization technique to compensate (because we haven't covered regularization in lecture yet.)

In [53]:
from sklearn.feature_extraction.text import CountVectorizer
model = Pipeline([
    ("SelectColumns", ColumnTransformer([
        ("keep", "passthrough", quantitative_features),
        ("cyl_vol", volume_transformer, ["cylinders", "displacement"]),
        ("text", CountVectorizer(), "name")
    ])),
    ("Imputation", SimpleImputer()),
    ("LinearModel", LinearRegression())
])

**Notice**: That we are using an additional column transformation.

In [54]:
model.fit(tr, tr['mpg'])
models['c+d+d/c+h+w+a+y+n'] = model

In [55]:
fig = compare_models(models)
fig.update_yaxes(range=[0,5.1], title="RMSE")

**Overfitting!**.  We substantially reduced the training error but actually made the generalization error worse!

In [56]:
best_model = clone(models['c+d+d/c+h+w+a+y'])

In [57]:
best_model.fit(data, data['mpg'])

Pipeline(steps=[('SelectColumns',
                 ColumnTransformer(transformers=[('keep', 'passthrough',
                                                  ['cylinders', 'displacement',
                                                   'horsepower', 'weight',
                                                   'acceleration',
                                                   'model_year']),
                                                 ('cyl_vol',
                                                  FunctionTransformer(func=<function compute_volume at 0x7fc7e77d7430>,
                                                                      validate=True),
                                                  ['cylinders',
                                                   'displacement'])])),
                ('Imputation', SimpleImputer()),
                ('LinearModel', LinearRegression())])

<br/><br/><br/>

---

## Adding the Text Features

### Adding the Origin

The origin is a categorical feature which takes on only a few values:

In [62]:
tr['origin'].value_counts()

usa       222
japan      71
europe     65
Name: origin, dtype: int64

This can be extracted using a `OneHotEncoder` in the `SelectColumns` stage of the pipeline. Notice that the `["origin"]` is in brackets, this caused me some confusion when preparing lecture.  The `OneHotEncoder` wants its input in column form rather than list form. 

In [63]:
from sklearn.preprocessing import OneHotEncoder

In [64]:
model = Pipeline([
    ("SelectColumns", ColumnTransformer([
        ("keep", "passthrough", quantitative_features),
        ("origin_encoder", OneHotEncoder(), ["origin"])
    ])),
    ("Imputation", SimpleImputer()),
    ("LinearModel", LinearRegression())
])

In [65]:
model.fit(tr, tr['mpg'])
name = ",".join([name[0] for name in quantitative_features]) + ",o"
models[name] = model

### Adding the Vehicle Name

In the previous lecture we added the vehicle name using the `CountVectorizer` which implements a bag-of-words encoding. 

In [66]:
from sklearn.feature_extraction.text import CountVectorizer

In [67]:
model = Pipeline([
    ("SelectColumns", ColumnTransformer([
        ("keep", "passthrough", quantitative_features),
        ("origin_encoder", OneHotEncoder(), ["origin"]),
        ("text", CountVectorizer(), "name")
    ])),
    ("Imputation", SimpleImputer()),
    ("LinearModel", LinearRegression())
])

In [68]:
model.fit(tr, tr['mpg'])
name = ",".join([name[0] for name in quantitative_features]) + ",o,n"
models[name] = model

In [69]:
compare_models(models)

Notice that adding the origin of the vehicle resulted in a small decrease in training error and validation error but adding the name resulted in a large decrease in training error and moderate increase in validation error.  The introduction of vehicle name feature resulted in overfitting.  To see why this might have happened let's look at the number of features in each model.

In [70]:
fig = go.Figure()
fig.add_trace(go.Bar(
    x = list(models.keys()), 
    y = [len(models[m]["LinearModel"].coef_) for m in models]
))
fig.update_yaxes(title="Number of Features",type="log")

Notice that the above plot is in log scale on the y axis!  The addition of the model name resulted in a large jump in features.  Perhaps some of these new features are useful and some are not.  What we really want now is a mechanism to select which of these features to keep and which to ignore or down-weight.  This can be done by using regularization. 

## Ridge Regression in SK Learn

Both Ridge Regression and Lasso are built-in functions in SKLearn.  Let's start by importing the `Ridge` Regression class which behaves identically to the `LinearRegression` class we used earlier:

In [82]:
from sklearn.linear_model import Ridge
from sklearn.model_selection import cross_val_score

Take a look at the documentation.  Notice the regularized loss function. 

In [73]:
Ridge?

We can plug the Ridge Regression Model in place of our the LinearRegression Model in our earlier pipeline.

In [83]:
ridge_model = Pipeline([
    ("SelectColumns", ColumnTransformer([
        ("keep", "passthrough", quantitative_features),
        ("origin_encoder", OneHotEncoder(), ["origin"]),
        ("text", CountVectorizer(), "name")
    ])),
    ("Imputation", SimpleImputer()),
    ("LinearModel", Ridge(alpha=0.5))
])

In [84]:
ridge_model.fit(tr, tr['mpg'])
models["Ridge(alpha=0.5)"] = ridge_model
compare_models(models)

We should also standardize our features:

In [85]:
from sklearn.preprocessing import StandardScaler

In [86]:
ridge_model = Pipeline([
    ("SelectColumns", ColumnTransformer([
        ("keep", StandardScaler(), quantitative_features),
        ("origin_encoder", OneHotEncoder(), ["origin"]),
        ("text", CountVectorizer(), "name")
    ])),
    ("Imputation", SimpleImputer()),
#     ("Standarize", StandardScaler(with_mean=False)),
    ("LinearModel", Ridge(alpha=0.5))
])

In [87]:
ridge_model.fit(tr, tr['mpg'])
models["RidgeN(alpha=0.5)"] = ridge_model
compare_models(models)

Notice that as we introduce regularization we are reducing the cross validation error and also increasing the training error.

What about different $\alpha$ hyper-parameter values?

In [88]:
ridge_model = Pipeline([
    ("SelectColumns", ColumnTransformer([
        ("keep", StandardScaler(), quantitative_features),
        ("origin_encoder", OneHotEncoder(), ["origin"]),
        ("text", CountVectorizer(), "name")
    ])),
    ("Imputation", SimpleImputer()),
    ("LinearModel", Ridge(alpha=10))
])

In [89]:
ridge_model.fit(tr, tr['mpg'])
models["RidgeN(alpha=10)"] = ridge_model
compare_models(models)

That is too much regularization.  Let's use cross validation to pick the best value.

### Cross Validation to Tune Regularization Parameter

The following uses cross validation to tune the regularization parameter.

In [106]:
def rmse_score(model, X, y):
    return np.sqrt(np.mean((y - model.predict(X))**2))

ridge_model = Pipeline([
    ("SelectColumns", ColumnTransformer([
        ("keep", StandardScaler(), quantitative_features),
        ("origin_encoder", OneHotEncoder(), ["origin"]),
        ("text", CountVectorizer(), "name")
    ])),
    ("Imputation", SimpleImputer()),
    ("LinearModel", Ridge(alpha=10))
])

alphas = np.linspace(0.5, 20, 30)
cv_values = []
train_values = []
test_values = []
for alpha in alphas:
    ridge_model.set_params(LinearModel__alpha=alpha)
    cv_values.append(np.mean(cross_val_score(ridge_model, tr, tr['mpg'], scoring=rmse_score, cv=5)))
    ridge_model.fit(tr, tr['mpg'])
    train_values.append(rmse_score(ridge_model, tr, tr['mpg']))
    test_values.append(rmse_score(ridge_model, te, te['mpg']))

We can plot the cross validation error against the different $\alpha$ values and pick the $\alpha$ with the smallest cross validation error. 

In [104]:
fig = go.Figure()
fig.add_trace(go.Scatter(x = alphas, y = train_values, mode="lines+markers", name="Train"))
fig.add_trace(go.Scatter(x = alphas, y = cv_values, mode="lines+markers", name="CV"))
fig.add_trace(go.Scatter(x = alphas, y = test_values, mode="lines+markers", name="Test"))
fig.update_layout(xaxis_title=r"$\alpha$", yaxis_title="CV RMSE")

Adding the best model:

In [107]:
best_alpha = alphas[np.argmin(cv_values)]
ridge_model.set_params(LinearModel__alpha=best_alpha)
ridge_model.fit(tr, tr['mpg'])
models["RidgeN(alpha_best)"] = ridge_model
compare_models(models)

We didn't have to do all of that work. SKLearn has a Ridge Regression class with built-in cross validation

In [108]:
from sklearn.linear_model import RidgeCV

In [109]:
alphas = np.linspace(0.5, 3, 30)

ridge_model = Pipeline([
    ("SelectColumns", ColumnTransformer([
        ("keep", StandardScaler(), quantitative_features),
        ("origin_encoder", OneHotEncoder(), ["origin"]),
        ("text", CountVectorizer(), "name")
    ])),
    ("Imputation", SimpleImputer()),
    ("LinearModel", RidgeCV(alphas=alphas))
])

In [110]:
ridge_model.fit(tr, tr['mpg'])
models["RidgeCV"] = ridge_model
compare_models(models)

## Lasso in SKLearn 

Similarly we can swap Ridge Regression for Lasso by simply importing the Lasso object from `sklearn.linear_model`.

In [97]:
from sklearn.linear_model import Lasso, LassoCV

In [98]:
lasso_model = Pipeline([
    ("SelectColumns", ColumnTransformer([
        ("keep", StandardScaler(), quantitative_features),
        ("origin_encoder", OneHotEncoder(), ["origin"]),
        ("text", CountVectorizer(), "name")
    ])),
    ("Imputation", SimpleImputer()),
    ("LinearModel", LassoCV(cv=3))
])

In [99]:
lasso_model.fit(tr, tr['mpg'])
models["LassoCV"] = lasso_model
compare_models(models)

We can examine the distribution of model coefficients to see that Lasso is selecting only a few of the features to use in it's predictions:

In [100]:
ff.create_distplot([
    models['LassoCV']["LinearModel"].coef_, 
    models['RidgeCV']["LinearModel"].coef_],
    ["Lasso", "Ridge"], bin_size=0.1)

Here we get the names of all the features in the model.

In [101]:
ct = models['LassoCV']['SelectColumns']
feature_names = (
    quantitative_features +
    list(ct.named_transformers_['origin_encoder'].get_feature_names())+
    list(ct.named_transformers_['text'].get_feature_names())
)
feature_names = np.array(feature_names)
print(feature_names)

['cylinders' 'displacement' 'horsepower' 'weight' 'acceleration'
 'model_year' 'x0_europe' 'x0_japan' 'x0_usa' '10' '100' '100ls' '111'
 '1131' '12' '1200' '1200d' '124' '124b' '128' '12tl' '1300' '131' '144ea'
 '1500' '1600' '18i' '1900' '200' '2000' '2002' '200sx' '210' '225' '2300'
 '240d' '244dl' '245' '264gl' '280' '280s' '2h' '300d' '304' '310' '320'
 '320i' '340' '350' '4000' '411' '4w' '500' '5000' '504' '505s' '510'
 '610' '626' '710' '810' '88' '99e' '99gle' '99le' 'accord' 'air'
 'ambassador' 'amc' 'aries' 'aspen' 'audi' 'auto' 'b210' 'beetle' 'bel'
 'benz' 'bmw' 'brougham' 'buick' 'c10' 'c20' 'cadillac' 'camaro' 'capri'
 'caprice' 'carina' 'carlo' 'catalina' 'cavalier' 'celica' 'century'
 'challenger' 'champ' 'charger' 'chevelle' 'chevette' 'chevroelt'
 'chevrolet' 'chevy' 'chrysler' 'ciera' 'citation' 'civic' 'classic'
 'cobra' 'colt' 'concord' 'concours' 'corolla' 'corona' 'coronet' 'cougar'
 'country' 'coupe' 'cressida' 'cricket' 'cruiser' 'cuda' 'custom'
 'cutlass' 'cvc

Finally, we select the features that had non-zero coefficients (parameters).  Not surprisingly, many of these features are likely good predictors of the fuel economy of a car.

In [102]:
kept = ~np.isclose(models['LassoCV']["LinearModel"].coef_, 0)
feature_names[kept]

array(['cylinders', 'weight', 'model_year', 'x0_japan', 'x0_usa', 'amc',
       'datsun', 'diesel', 'ford', 'honda', 'horizon', 'ii', 'plymouth',
       'pontiac', 'volvo', 'vw'], dtype='<U12')